In [20]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from resources import Plots
from environment.Config import ConfigTimeSeries

def binaryState(timeseries, cursor, action):
    """
    :param timeseries:
    :param cursor: the position where in the TimeSeries we are currently
    :return: The Value of the current position, states with the same value are treated the same way
    """
    state = np.asarray([np.float64(timeseries['value'][cursor]), 1 if action == 1 else 0])
    return state


def binaryReward(state, timeseries, cursor, action, path=None):
    if state[1] == 1 and timeseries['anomaly'][cursor] == 1:
        if action == 1:
            return 1
        if action == 0:
            return -1
    if state[1] == 1 and timeseries['anomaly'][cursor] == 0:
        if action == 1:
            return -1
        if action == 0:
            return 1
    if state[1] == 0 and timeseries['anomaly'][cursor] == 1:
        if action == 1:
            return 1
        if action == 0:
            return -1
    if state[1] == 0 and timeseries['anomaly'][cursor] == 0:
        if action == 1:
            return -1
        if action == 0:
            return 1


In [21]:
class TimeSeriesEnvironment:
    def __init__(self, directory="../ts_data/",
                 config=ConfigTimeSeries(normal=0, anomaly=1, reward_correct=1, reward_incorrect=-1,
                                         action_space=[0, 1], seperator=",", boosted=False),
                 filename="Test/SmallData.csv", verbose=False,
                 scaler=MinMaxScaler(), window=False):
        """
        Initialization of one TimeSeries, should have suitable methods to prepare DataScience
        :param directory: Path to The Datasets
        :param config: Config class Object with specified Reward Settings
        :param filename: The Name of the specified TimeSeries we are working on
        :param verbose: if True, then prints __str__
        """
        self.filename = filename
        self.file = os.path.join(directory + self.filename)
        self.cfg = config
        self.sep = config.seperator
        self.boosted = config.boosted
        self.window = window
        self.scaler = scaler

        self.action_space_n = len(self.cfg.action_space)

        self.timeseries_cursor = -1
        self.timeseries_cursor_init = 0
        self.timeseries_states = []
        self.done = False

        self.statefunction = binaryState
        self.rewardfunction = binaryReward
        self.isdone = False

        self.timeseries_labeled = pd.read_csv(self.file, usecols=[1, 2], header=0, sep=self.sep,
                                              names=['value', 'anomaly'],
                                              encoding="utf-8")

        self.timeseries_unlabeled = pd.read_csv(self.file, usecols=[1], header=0, sep=self.sep,
                                                names=['value'],
                                                encoding="utf-8")

        if verbose:
            print(self.__str__())

    def __str__(self):
        """
        :return: String Representation of the TimeSeriesEnvironment Class, mainly for debug information
        """
        return "TimeSeries from: {}\n Header(labeled):\n {} \nHeader(unlabeled):\n {} \nRows:\n " \
               "{}\nMeanValue:\n {}\nMaxValue:\n {}\nMinValue:\n {}".format(
            self.filename,
            self.timeseries_labeled.head(
                3),
            self.timeseries_unlabeled.head(
                3),
            self.timeseries_labeled.shape[0],
            round(self.timeseries_labeled["value"].mean(), 2),
            round(self.timeseries_labeled["value"].max(), 2),
            round(self.timeseries_labeled["value"].min(), 2))

    # reset the environment
    def reset(self, cursor_init=0):
        self.timeseries_cursor = cursor_init
        self.done = False
        self.normalize_timeseries()
        if not self.window:
            init_state = self.statefunction(self.timeseries_cursor)
        else:
            init_state = self.statefunction(self.timeseries_labeled, self.timeseries_cursor, 0)
        return init_state

    # take a step and gain a reward
    def step(self, action):
        current_state = self.statefunction(self.timeseries_labeled, self.timeseries_cursor, action)
        action_in = action
        reward = self.rewardfunction(current_state, self.timeseries_labeled, self.timeseries_cursor, action)
        self.update_cursor()
        next_state = self.statefunction(self.timeseries_labeled, self.timeseries_cursor, action)
        return current_state, action, reward, next_state, self.is_done(self.timeseries_cursor)

    def update_cursor(self):
        self.timeseries_cursor += 1

    def is_done(self, cursor):
        if cursor >= len(self.timeseries_labeled) - 1:
            self.done = True
            return True
        else:
            self.done = False
            return False

    def normalize_timeseries(self):
        self.timeseries_labeled["value"] = self.scaler.fit_transform(self.timeseries_labeled[["value"]])

    def isanomaly(self, cursor):
        if self.timeseries_labeled['anomaly'][cursor] == 1:
            return 1
        else:
            return 0

    def get_series(self, labelled=True):
        if labelled:
            return self.timeseries_labeled
        return self.timeseries_unlabeled

    def get_name(self):
        return self.filename

In [22]:
config = ConfigTimeSeries(seperator=",", window=1)
env = TimeSeriesEnvironment(verbose=True, filename="./Test/SmallData_1.csv", config=config, window=True)

env.reset()
count = 0
env.normalize_timeseries()
while True:
    current_state, action, reward, next_state, done = env.step(1)
    print(current_state, action, reward, next_state, done)
    if done:
        break


TimeSeries from: ./Test/SmallData_1.csv
 Header(labeled):
    value  anomaly
0      0        0
1      0        0
2      4        0 
Header(unlabeled):
    value
0      0
1      0
2      4 
Rows:
 1424
MeanValue:
 40.39
MaxValue:
 452
MinValue:
 0
[0. 1.] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00884956 1.        ] False
[0.00884956 1.        ] 1 -1 [0.00442478 1.        ] False
[0.00442478 1.        ] 1 -1 [0.00884956 1.        ] False
[0.00884956 1.        ] 1 -1 [0.01548673 1.        ] False
[0.01548673 1.        ] 1 -1 [0.02212389 1.        ] False
[0.02212389 1.        ] 1 -1 [0.07964602 1.        ] False
[0.07964602 1.        ] 1 -1 [0.12610619 1.        ] False
[0.12610619 1.        ] 1 -1 [0.16371681 1.        ] False
[0.16371681 1.        ] 1 -1 [0.16150442 1.        ] False
[0.16150442 1.        ] 1 -1 [0.13495575 1.        ] False
[0.13495575 1.        ] 1 -1 [0.0420354 1.       ] False
[0.0420354 1.       ] 1 -1 [0.02654867 1.        ] False
[0.02654867 1.        ] 1 -1 [0.022123

C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


 False
[0.01769912 1.        ] 1 -1 [0.03539823 1.        ] False
[0.03539823 1.        ] 1 -1 [0.03539823 1.        ] False
[0.03539823 1.        ] 1 -1 [0.03097345 1.        ] False
[0.03097345 1.        ] 1 -1 [0.04867257 1.        ] False
[0.04867257 1.        ] 1 -1 [0.11283186 1.        ] False
[0.11283186 1.        ] 1 -1 [0.13053097 1.        ] False
[0.13053097 1.        ] 1 -1 [0.15044248 1.        ] False
[0.15044248 1.        ] 1 -1 [0.14380531 1.        ] False
[0.14380531 1.        ] 1 -1 [0.14823009 1.        ] False
[0.14823009 1.        ] 1 -1 [0.1460177 1.       ] False
[0.1460177 1.       ] 1 -1 [0.15044248 1.        ] False
[0.15044248 1.        ] 1 -1 [0.04867257 1.        ] False
[0.04867257 1.        ] 1 -1 [0.09734513 1.        ] False
[0.09734513 1.        ] 1 -1 [0.02654867 1.        ] False
[0.02654867 1.        ] 1 -1 [0.05088496 1.        ] False
[0.05088496 1.        ] 1 -1 [0.02433628 1.        ] False
[0.02433628 1.        ] 1 -1 [0.00884956 1.        ] 

[0.03318584 1.        ] 1 -1 [0.03982301 1.        ] False
[0.03982301 1.        ] 1 -1 [0.04424779 1.        ] False
[0.04424779 1.        ] 1 -1 [0.06415929 1.        ] False
[0.06415929 1.        ] 1 -1 [0.14823009 1.        ] False
[0.14823009 1.        ] 1 -1 [0.13716814 1.        ] False
[0.13716814 1.        ] 1 -1 [0.15044248 1.        ] False
[0.15044248 1.        ] 1 -1 [0.13053097 1.        ] False
[0.13053097 1.        ] 1 -1 [0.12168142 1.        ] False
[0.12168142 1.        ] 1 -1 [0.13938053 1.        ] False
[0.13938053 1.        ] 1 -1 [0.12168142 1.        ] False
[0.12168142 1.        ] 1 -1 [0.11946903 1.        ] False
[0.11946903 1.        ] 1 -1 [0.11946903 1.        ] False
[0.11946903 1.        ] 1 -1 [0.11946903 1.        ] False
[0.11946903 1.        ] 1 -1 [0.03539823 1.        ] False
[0.03539823 1.        ] 1 -1 [0.01106195 1.        ] False
[0.01106195 1.        ] 1 -1 [0.0420354 1.       ] False
[0.0420354 1.       ] 1 -1 [0.01106195 1.        ] False
[

[0.24557522 1.        ] 1 1 [0.20353982 1.        ] False
[0.20353982 1.        ] 1 1 [0.2079646 1.       ] False
[0.2079646 1.       ] 1 1 [0.16150442 1.        ] False
[0.16150442 1.        ] 1 1 [0.19911504 1.        ] False
[0.19911504 1.        ] 1 1 [0.18141593 1.        ] False
[0.18141593 1.        ] 1 1 [0.2920354 1.       ] False
[0.2920354 1.       ] 1 1 [0.34734513 1.        ] False
[0.34734513 1.        ] 1 1 [0.5420354 1.       ] False
[0.5420354 1.       ] 1 1 [0.55530973 1.        ] False
[0.55530973 1.        ] 1 1 [0.4579646 1.       ] False
[0.4579646 1.       ] 1 1 [0.51327434 1.        ] False
[0.51327434 1.        ] 1 1 [0.64380531 1.        ] False
[0.64380531 1.        ] 1 1 [0.5 1. ] False
[0.5 1. ] 1 1 [0.45353982 1.        ] False
[0.45353982 1.        ] 1 1 [0.44247788 1.        ] False
[0.44247788 1.        ] 1 1 [0.37831858 1.        ] False
[0.37831858 1.        ] 1 1 [0.71238938 1.        ] False
[0.71238938 1.        ] 1 1 [0.50663717 1.        ] False


[0.0840708 1.       ] 1 -1 [0.05309735 1.        ] False
[0.05309735 1.        ] 1 -1 [0.02654867 1.        ] False
[0.02654867 1.        ] 1 -1 [0.0199115 1.       ] False
[0.0199115 1.       ] 1 -1 [0.00663717 1.        ] False
[0.00663717 1.        ] 1 -1 [0.03539823 1.        ] False
[0.03539823 1.        ] 1 -1 [0.05088496 1.        ] False
[0.05088496 1.        ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.0199115 1.       ] False
[0.0199115 1.       ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00884956 1.        ] False
[0.00884956 1.        ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00663717 1.        ] False
[0.00663717 1.        ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00442478 1.        ] False
[0.00442478 1.        ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00884956 1.        ] False
[0.00884956 1.        ] 1 -1 [0. 1.] False
[0. 1.] 1 -1 [0.00663717 1.        ] False
[0.00663717 1.        ] 1 -1 [0.00442478 1.        ] False
[0.00442478 1.        ] 1 -1 [0.02433628 1.        ] False
[0.02433628 1.        